In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%pip install kagglehub

In [3]:
import kagglehub

path = kagglehub.dataset_download("robertocarlost/fmcg-multi-country-sales-dataset")

print("Path to dataset files:", path)

100%|██████████| 29.7M/29.7M [00:00<00:00, 107MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/robertocarlost/fmcg-multi-country-sales-dataset/versions/1


In [4]:
print("Files in dataset:", os.listdir(path))

df = pd.read_csv(os.path.join(path, "fmcg_sales_3years_1M_rows.csv"))
df.head()

Files in dataset: ['fmcg_sales_3years_1M_rows.csv']


,date,year,month,day,weekofyear,weekday,is_weekend,is_holiday,temperature,rain_mm,...,discount_pct,promo_flag,gross_sales,net_sales,stock_on_hand,stock_out_flag,lead_time_days,supplier_id,purchase_cost,margin_pct
0,2021-01-01,2021,1,1,53,4,0,1,8.44,1.24,...,0.1,1,167.84,151.06,248,0,11,S008,7.53,0.182
1,2021-01-02,2021,1,2,53,5,1,0,12.61,1.12,...,0.0,0,125.88,125.88,238,0,6,S057,5.19,0.505
2,2021-01-03,2021,1,3,53,6,1,0,12.02,2.69,...,0.3,1,398.62,279.03,238,0,6,S017,5.59,0.168
3,2021-01-04,2021,1,4,1,0,0,0,7.76,4.65,...,0.0,0,83.92,83.92,216,0,7,S012,7.81,0.255
4,2021-01-05,2021,1,5,1,1,0,0,11.16,1.77,...,0.2,1,178.33,142.66,372,0,8,S038,7.62,0.073


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100000 entries, 0 to 1099999
Data columns (total 33 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   date            1100000 non-null  object 
 1   year            1100000 non-null  int64  
 2   month           1100000 non-null  int64  
 3   day             1100000 non-null  int64  
 4   weekofyear      1100000 non-null  int64  
 5   weekday         1100000 non-null  int64  
 6   is_weekend      1100000 non-null  int64  
 7   is_holiday      1100000 non-null  int64  
 8   temperature     1100000 non-null  float64
 9   rain_mm         1100000 non-null  float64
 10  store_id        1100000 non-null  object 
 11  country         1100000 non-null  object 
 12  city            1100000 non-null  object 
 13  channel         1100000 non-null  object 
 14  latitude        1100000 non-null  float64
 15  longitude       1100000 non-null  float64
 16  sku_id          1100000 non-null  ob

### Konversi tipe data kolom date menjadi datetime
Tipe data sebelum konversi: object
Tipe data setelah konversi: datetime64[ns]

Urgensi:
Google looker tidak dapat mengenali kolom object sebagai tanggal. Tanpa melakukan konversi ini, Google Looker tidak akan dapat menampilkan data tanggal dengan benar seperti tidak dapat membuat time-series chart, menggunakan date filter (calendar picker), melakukan perbandingan YoY/MoM, atau menggunakan built-in function seperti time-based aggregation.

In [6]:
df['date'] = pd.to_datetime(df['date'])

### Identifikasi dan Penanganan Data Duplikat
Identifikasi dan penanganan data duplikat diperlukan karena jika ada data duplikat bisa menyebabkan double-counting pada agregasi. Sebagai contoh pada project ini, jika ada satu transaksi yang tercatat dua kali maka SUM(net_sales) akan menghitung dua kali lipat sehingga membuat kesalahan fatal dalam analisis bisnis.   

In [7]:
print(f"Duplikat: {df.duplicated().sum()}")
df = df.drop_duplicates()

Duplikat: 0


### Validasi Konsistensi Data
Validasi konsistensi data diperlukan sebab data yang tidak logis akan menghasilkan insight yang salah. Sebagai contoh jika net_sales > gross_sales, maka ada kesalahan data karena mustahil penjualan bersih (net_sales) lebih besar dari penjualan kotor (gross_sales). 

Validasi yang akan dilakukan adalah:
1. net_sales <= gross_sales, Net = Gross - Discount
2. discount_pct (antara 0-1), Discount tidak bisa negatif atau > 100%
3. units_sold >= 0, Tidak ada penjualan negatif
4. margin_pct (antara 0-1), Margin harus valid

In [8]:
print("net_sales > gross_sales:", (df['net_sales'] > df['gross_sales']).sum())
print("Invalid discount:", ((df['discount_pct'] < 0) | (df['discount_pct'] > 1)).sum())
print("Negative units:", (df['units_sold'] < 0).sum())

net_sales > gross_sales: 0
Invalid discount: 0
Negative units: 0


### Agregasi Data
Data pada dataset berjumlah sebanyak 1.1 juta baris, jumlah ini terlalu besar untuk google looker. Oleh karena itu, diperlukan agregasi data untuk menghindari:
1. Timeout saat query
2. Quota exceeded di Google Cloude
3. Visualisasi menjadi lambat
4. Crash Browser saat render chart

Agregasi data yang akan dilakukan:
Mengelompokkan data berdasarkan tanggal, negara, kategori, dan channel. Lalu menghitung ringkasan angka untuk setiap grup, seperti menjumlahkan units_sold, gross_sales, net_sales, dan purchase_cost, serta mengambil rata-rata stock_on_hand. Setelah itu hasil yang didapatkan dikembalikan menjadi tabel biasa dengan reset_index sehingga menghasilkan DataFrame baru berisi data yang sudah agregasi berdasarkan tanggal, negara, kategori, dan channel.

In [9]:
df_agg = df.groupby(['date', 'country', 'city', 'category', 'channel']).agg({
    'units_sold': 'sum',
    'gross_sales': 'sum',
    'net_sales': 'sum',
    'stock_on_hand': 'mean',
    'purchase_cost': 'sum'
}).reset_index()

df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65700 entries, 0 to 65699
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           65700 non-null  datetime64[ns]
 1   country        65700 non-null  object        
 2   city           65700 non-null  object        
 3   category       65700 non-null  object        
 4   channel        65700 non-null  object        
 5   units_sold     65700 non-null  int64         
 6   gross_sales    65700 non-null  float64       
 7   net_sales      65700 non-null  float64       
 8   stock_on_hand  65700 non-null  float64       
 9   purchase_cost  65700 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 5.0+ MB


### Deteksi Outlier
Deteksi outlier tetap dilakukan menggunakan metode IQR (Interquartile Range). Namun, Outlier handling tidak dilakukan sebab sales data merupakan data transaksional rill (nilai yang tinggi bisa jadi legitimate: bulk order, seasonal peak, promosi besar, pelanggan korporat), data sudah diagregasi (nilai ekstrem pada df_agg mencerminkan akumulasi riil, bukan error pengukuran), menghapus data dapat menimbulkan kehilangan bisnis insight (outlier tinggi bisa menunjukkan best-selling product/store yang justru penting dianalisis).

In [10]:
Q1, Q3 = df_agg['net_sales'].quantile([0.25, 0.75])
IQR = Q3 - Q1
outliers = df_agg[(df_agg['net_sales'] < Q1-1.5*IQR) | (df_agg['net_sales'] > Q3+1.5*IQR)]
print(f"Outliers: {len(outliers)}")

Outliers: 3682


### Segmentasi Data
Segmentasi penjualan dilakukan dengan membuat kolom baru bernama sales_segment berdasarkan nilai net_sales. Nilai penjualan dikelompokkan ke dalam beberapa kategori yang telah ditentukan, yaitu: 
1. 0–100 sebagai Micro
2. 100–500 sebagai Small
3. 500–2000 sebagai Medium
4. 2000–10000 sebagai Large
5. lebih dari 10000 sebagai Enterprise.

Urgensi:
Untuk memudahkan analisis berdasarkan ukuran transaksi, sehingga outliers akan masuk kedalam kategori Enterprise dan tetap dapat dianalisis, memudahkan segmentasi di google looker, dan chart pie atau bar berdasarkan segment dapat lebih informatif.

In [11]:
df_agg['sales_segment'] = pd.cut(
    df_agg['net_sales'],
    bins=[0, 100, 500, 2000, 10000, float('inf')],
    labels=['Micro', 'Small', 'Medium', 'Large', 'Enterprise']
)

In [12]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65700 entries, 0 to 65699
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           65700 non-null  datetime64[ns]
 1   country        65700 non-null  object        
 2   city           65700 non-null  object        
 3   category       65700 non-null  object        
 4   channel        65700 non-null  object        
 5   units_sold     65700 non-null  int64         
 6   gross_sales    65700 non-null  float64       
 7   net_sales      65700 non-null  float64       
 8   stock_on_hand  65700 non-null  float64       
 9   purchase_cost  65700 non-null  float64       
 10  sales_segment  65700 non-null  category      
dtypes: category(1), datetime64[ns](1), float64(4), int64(1), object(4)
memory usage: 5.1+ MB


### Standardisasi Text Columns
Standardisasi text columns dilakukan untuk membuat data lebih rapi, konsisten, dan siap digunakan untuk analisis. Standardisasi ini melibatkan beberapa langkah:
1. Menghapus spasi berlebih di awal dan akhir teks, sehingga nilai seperti " indonesia " menjadi "indonesia".
2. Mengubah format teks menjadi Title Case (huruf pertama tiap kata menjadi kapital), misalnya "united states" menjadi "United States".

In [13]:
df_agg['country'] = df_agg['country'].str.strip().str.title()
df_agg['city'] = df_agg['city'].str.strip().str.title()
df_agg['category'] = df_agg['category'].str.strip().str.title()

df_agg.head()

,date,country,city,category,channel,units_sold,gross_sales,net_sales,stock_on_hand,purchase_cost,sales_segment
0,2021-01-01,Austria,Vienna,Beverages,Hypermarket,1301,9158.19,8344.67,332.000000,79.38,Large
1,2021-01-01,Austria,Vienna,Dairy,Hypermarket,775,6983.17,6983.17,298.416667,57.41,Large
2,2021-01-01,Austria,Vienna,Home Care,Hypermarket,654,4047.40,4047.40,297.833333,44.73,Large
3,2021-01-01,Austria,Vienna,Personal Care,Hypermarket,1112,9531.09,9329.75,319.166667,98.85,Large
4,2021-01-01,Austria,Vienna,Snacks,Hypermarket,1167,9713.71,9680.78,307.526316,88.16,Large


### Feature Engineering
Rekayasa fitur dilakukan untuk menambahkan kolom baru ke dalam dataset guna mempermudah melakukan visualisasi data berdasarkan keuntungan dan data pada kuartal tertentu. Kolom baru yang ditambahkan adalah "profit" dan "quarter".

In [14]:
df_agg['profit'] = df['net_sales'] - (df['units_sold'] * df['purchase_cost'])
df_agg['quarter'] = ((df['month'] - 1) // 3) + 1

df_agg

,date,country,city,category,channel,units_sold,gross_sales,net_sales,stock_on_hand,purchase_cost,sales_segment,profit,quarter
0,2021-01-01,Austria,Vienna,Beverages,Hypermarket,1301,9158.19,8344.67,332.000000,79.38,Large,30.58,1
1,2021-01-01,Austria,Vienna,Dairy,Hypermarket,775,6983.17,6983.17,298.416667,57.41,Large,63.60,1
2,2021-01-01,Austria,Vienna,Home Care,Hypermarket,654,4047.40,4047.40,297.833333,44.73,Large,66.61,1
3,2021-01-01,Austria,Vienna,Personal Care,Hypermarket,1112,9531.09,9329.75,319.166667,98.85,Large,21.44,1
4,2021-01-01,Austria,Vienna,Snacks,Hypermarket,1167,9713.71,9680.78,307.526316,88.16,Large,13.12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65695,2023-12-31,Spain,Madrid,Beverages,E-commerce,1178,7914.65,7914.65,270.631579,77.16,Large,155.35,4
65696,2023-12-31,Spain,Madrid,Dairy,E-commerce,798,6999.83,6967.82,296.133333,72.42,Large,230.23,4
65697,2023-12-31,Spain,Madrid,Home Care,E-commerce,705,5151.45,5151.45,309.153846,65.99,Large,109.06,4
65698,2023-12-31,Spain,Madrid,Personal Care,E-commerce,629,6027.59,5982.58,305.357143,78.02,Large,206.80,4


In [15]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65700 entries, 0 to 65699
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           65700 non-null  datetime64[ns]
 1   country        65700 non-null  object        
 2   city           65700 non-null  object        
 3   category       65700 non-null  object        
 4   channel        65700 non-null  object        
 5   units_sold     65700 non-null  int64         
 6   gross_sales    65700 non-null  float64       
 7   net_sales      65700 non-null  float64       
 8   stock_on_hand  65700 non-null  float64       
 9   purchase_cost  65700 non-null  float64       
 10  sales_segment  65700 non-null  category      
 11  profit         65700 non-null  float64       
 12  quarter        65700 non-null  int64         
dtypes: category(1), datetime64[ns](1), float64(5), int64(2), object(4)
memory usage: 6.1+ MB
